In [ ]:
# Adapted from:
# https://github.com/t-vi/pytorch-tvmisc/blob/master/visualize/bad_grad_viz.ipynb
# I've added the grad norm at each edge
from graphviz import Digraph
import mup
import torch
from torch.autograd import Variable, Function

import babylm.models
from transformers import CONFIG_MAPPING, AutoModelForCausalLM, AutoTokenizer


def iter_graph(root, callback):
    queue = [root]
    seen = set()
    while queue:
        fn = queue.pop()
        if fn in seen:
            continue
        seen.add(fn)
        for next_fn, _ in fn.next_functions:
            if next_fn is not None:
                queue.append(next_fn)
        callback(fn)

def register_hooks(var):
    fn_dict = {}
    def hook_cb(fn):
        def register_grad(grad_input, grad_output):
            fn_dict[fn] = grad_input
        fn.register_hook(register_grad)
    iter_graph(var.grad_fn, hook_cb)

    def is_bad_grad(grad_output):
        if grad_output is None:
            return False
        return grad_output.isnan().any() or (grad_output.abs() >= 1e6).any()

    def make_dot():
        node_attr = dict(style='filled',
                        shape='box',
                        align='left',
                        fontsize='12',
                        ranksep='0.1',
                        height='0.2')
        dot = Digraph(node_attr=node_attr, graph_attr=dict(size="12,12"))

        def size_to_str(size):
            return '('+(', ').join(map(str, size))+')'

        def build_graph(fn):
            if hasattr(fn, 'variable'):  # if GradAccumulator
                u = fn.variable
                node_name = 'Variable\n ' + size_to_str(u.size())
                dot.node(str(id(u)), node_name, fillcolor='lightblue')
            else:
                assert fn in fn_dict, fn
                fillcolor = 'white'
                if any(is_bad_grad(gi) for gi in fn_dict[fn]):
                    fillcolor = 'red'
                dot.node(str(id(fn)), str(type(fn).__name__), fillcolor=fillcolor)
            for grad_input, (next_fn, _) in zip(fn_dict[fn], fn.next_functions):
                if grad_input is not None:
                    grad_mag = torch.linalg.norm(grad_input)
                else:
                    grad_mag = 0
                if next_fn is not None:
                    next_id = id(getattr(next_fn, 'variable', next_fn))
                    dot.edge(str(next_id), str(id(fn)), label=f'{grad_mag:.6f}')
        iter_graph(var.grad_fn, build_graph)

        return dot

    return make_dot



path = './outputs'
# model = AutoModelForCausalLM.from_pretrained(path)
config = CONFIG_MAPPING['holo']()
model = AutoModelForCausalLM.from_config(config)
base_shapes = model.mup_base_shapes()
mup.set_base_shapes(model, base_shapes)
tokenizer = AutoTokenizer.from_pretrained('gpt2')

num_batches = 2
seq_len = 1024

tokens = torch.randint(1, tokenizer.vocab_size, (num_batches, seq_len))
output = model(tokens, labels=tokens)

get_dot = register_hooks(output.loss)
output.loss.backward()
dot = get_dot()
#dot.save('tmp.dot') # to get .dot
#dot.render('tmp') # to get SVG
dot # in Jupyter, you can just render the variable